## Imports

In [1]:
import pandas as pd
from cred_helper import set_aws_credentials, txt2dict
from s3_helper import S3Helper
from loguru import logger

### Set credentials as environment variables

In [2]:
set_aws_credentials(txt2dict('./config/local/aws_credentials.txt'))

### Get df with class S3Helper

In [3]:
s3h = S3Helper()
s3h.client()
df = s3h.load_df(bucket_name='ds-main-s3-200624937306-dataset', file_key='ds-mtc-dialdig/call_center_dia_tudo.xlsx', type='excel')

2023-05-31 15:35:49.396 | SUCCESS  | s3_helper.s3_helper:load_s3:23 - Função realizada com sucesso
/home/thiago/.conda/envs/le/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-05-31 15:35:49.605 | SUCCESS  | s3_helper.s3_helper:data2df:51 - Sucesso. Dataframe contém 398 linhas


### preprocessing

In [4]:
def rename_cols(df):
    key_mapping = {key: key.lower().replace(' ', '_').replace('%','porc_').replace('.','') for key in df.columns}
    df.rename(columns=key_mapping, inplace=True)

In [5]:
def dropna_cols(df, porc = 0.3):
    nans = df.isna().sum()/df.shape[0]
    for i in nans.index:
        if nans[i] > porc:
            df.drop(i, axis=1, inplace=True)
            logger.info('Coluna {} removida por ter mais de {}% de valores nulos'.format(i, porc*100))

In [6]:
rename_cols(df)
df = df.dropna()
df = df.iloc[:-1,:]
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d %H:%M:%S')
df.isna().sum()

data                       0
cham_forecast              0
cham_recebidas             0
cham_atendidas             0
cham_ativas                0
chat                       0
total_contatos             0
tma_ativo                  0
porc_fcr21                 0
cham_abandonadas           0
porc__abandono             0
porc__nível_de_serviço_    0
tma                        0
tme                        0
t_médio_abandono           0
porc__nss                  0
porc__pesq_respondidas     0
atendidas_ns               0
dtype: int64

### Exploratory analysis